In [1]:
import pandas as pd
from category_encoders import TargetEncoder

train_df = pd.read_csv("/Users/larry/house-price-regression/data/processed/train_cleaned.csv")
eval_df = pd.read_csv("/Users/larry/house-price-regression/data/processed/eval_cleaned.csv")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print(train_df["date"].min(), "to", train_df["date"].max())
print(eval_df["date"].min(), "to", eval_df["date"].max())

train_df["date"] = pd.to_datetime(train_df["date"])
eval_df["date"] = pd.to_datetime(eval_df["date"])

2012-03-31 to 2019-12-31
2020-01-31 to 2021-12-31


In [2]:
train_df.head(5)


,date,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,avg_sale_to_list,sold_above_list,off_market_in_two_weeks,city,zipcode,year,bank,bus,hospital,mall,park,restaurant,school,station,supermarket,Total Population,Median Age,Per Capita Income,Total Families Below Poverty,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full,lat,lng
0,2012-03-31,46550.0,217450.0,31.813674,110.183666,14.0,23.0,44.0,64.0,59.5,0.943662,0.142857,0.043478,ATL,30002,2012,12.0,2.0,4.0,1.0,60.0,45.0,57.0,4.0,7.0,5811.0,36.3,33052.0,5811.0,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,200773.999557,Atlanta-Sandy Springs-Roswell,33.7338,-84.3922
1,2012-03-31,200000.0,7500.0,104.931794,79.265873,1.0,1.0,1.0,2.0,290.0,0.909091,0.000000,0.000000,PGH,15469,2012,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2441.0,41.8,20241.0,2385.0,1108.0,641.0,94600.0,1171.0,52.0,2376.0,2376.0,1018.0,105863.681174,Pittsburgh,40.4744,-79.8632
2,2012-03-31,196000.0,198000.0,122.807018,124.060150,1.0,1.0,1.0,1.0,49.0,0.989899,0.000000,0.000000,PGH,15464,2012,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,1515.0,45.1,18623.0,1508.0,692.0,484.0,79300.0,711.0,61.0,1489.0,1489.0,641.0,126572.277873,Pittsburgh,40.4744,-79.8632
3,2012-03-31,340000.0,349900.0,171.817343,242.367722,13.0,18.0,30.0,54.0,126.0,0.901457,0.153846,0.277778,BOS,2739,2012,0.0,0.0,0.0,0.0,10.0,0.0,14.0,0.0,0.0,6073.0,47.4,39312.0,6055.0,3324.0,1142.0,419600.0,3207.0,147.0,5982.0,5982.0,2794.0,352711.838012,Boston-Cambridge-Newton,42.5584,-71.1007
4,2012-03-31,51770.0,66950.0,68.638393,32.495345,7.0,7.0,6.0,19.0,111.0,0.887859,0.142857,0.000000,PGH,15461,2012,0.0,0.0,0.0,0.0,1.0,0.0,5.0,0.0,0.0,4191.0,42.1,19105.0,4158.0,2033.0,501.0,72100.0,1706.0,175.0,4037.0,4037.0,1489.0,70415.266483,Pittsburgh,40.4744,-79.8632


In [3]:
def date_feature(df: pd.DataFrame) -> pd.DataFrame:
    """
    Splits the date column into individual year, month, and day columns
    and inserts them into the dataframe in the order of year, month, day.
    """
    df["year"] = df["date"].dt.year
    df["month"] = df["date"].dt.month
    df["day"] = df["date"].dt.day

    df.insert(1, "year", df.pop("year"))
    df.insert(2, "month", df.pop("month"))
    df.insert(3, "day", df.pop("day"))
    return df

In [4]:
train_df = date_feature(train_df)
eval_df = date_feature(eval_df)

train_df.head(1)

,date,year,month,day,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,avg_sale_to_list,sold_above_list,off_market_in_two_weeks,city,zipcode,bank,bus,hospital,mall,park,restaurant,school,station,supermarket,Total Population,Median Age,Per Capita Income,Total Families Below Poverty,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full,lat,lng
0,2012-03-31,2012,3,31,46550.0,217450.0,31.813674,110.183666,14.0,23.0,44.0,64.0,59.5,0.943662,0.142857,0.043478,ATL,30002,12.0,2.0,4.0,1.0,60.0,45.0,57.0,4.0,7.0,5811.0,36.3,33052.0,5811.0,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,200773.999557,Atlanta-Sandy Springs-Roswell,33.7338,-84.3922


In [5]:
eval_df.head(2)

,date,year,month,day,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,avg_sale_to_list,sold_above_list,off_market_in_two_weeks,city,zipcode,bank,bus,hospital,mall,park,restaurant,school,station,supermarket,Total Population,Median Age,Per Capita Income,Total Families Below Poverty,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full,lat,lng
0,2020-01-31,2020,1,31,184000.0,168450.0,123.333333,109.899800,35.0,30.0,34.0,37.0,50.0,0.954732,0.171429,0.300000,CHI,60482,3.0,0.0,2.0,0.0,68.0,30.0,60.0,5.0,14.0,10849.0,42.1,27037.0,10795.0,4598.0,964.0,169000.0,5690.0,414.0,10573.0,10573.0,5089.0,164839.734849,Chicago-Naperville-Elgin,41.6675,-87.9597
1,2020-01-31,2020,1,31,140000.0,139000.0,94.691957,111.311054,61.0,81.0,83.0,54.0,10.0,0.963039,0.098361,0.469136,CIN,41071,35.0,5.0,4.0,3.0,139.0,259.0,86.0,1.0,8.0,20624.0,36.6,30324.0,19423.0,10164.0,802.0,132400.0,10479.0,552.0,19809.0,19809.0,9383.0,132164.314962,Cincinnati,39.0811,-84.4646


In [6]:
"""
Frequency Encoding:
 - For Zipcodes
 - For each zipcode, calculate the frequency of that zipcode in the training data 
   and replace the zipcode with that frequency.
"""

# Create frequency dict
zipcode_freq = train_df["zipcode"].value_counts() 

train_df["zipcode_freq"] = train_df["zipcode"].map(zipcode_freq)

# Don't compute frequency on evaluation
eval_df["zipcode_freq"] = eval_df["zipcode"].map(zipcode_freq).fillna(0)

print(train_df[["zipcode", "zipcode_freq"]].head())



   zipcode  zipcode_freq
0    30002            94
1    15469            86
2    15464            85
3     2739            94
4    15461            94


In [7]:
train_df.head(1)

,date,year,month,day,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,avg_sale_to_list,sold_above_list,off_market_in_two_weeks,city,zipcode,bank,bus,hospital,mall,park,restaurant,school,station,supermarket,Total Population,Median Age,Per Capita Income,Total Families Below Poverty,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full,lat,lng,zipcode_freq
0,2012-03-31,2012,3,31,46550.0,217450.0,31.813674,110.183666,14.0,23.0,44.0,64.0,59.5,0.943662,0.142857,0.043478,ATL,30002,12.0,2.0,4.0,1.0,60.0,45.0,57.0,4.0,7.0,5811.0,36.3,33052.0,5811.0,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,200773.999557,Atlanta-Sandy Springs-Roswell,33.7338,-84.3922,94


In [8]:
"""
Target Encoding:
For each city, encode the city with the avg price 
"""
te = TargetEncoder(cols=["city_full"])
train_df["encoded_city"] = te.fit_transform(train_df["city_full"], train_df["price"])
eval_df["encoded_city"] = te.transform(eval_df["city_full"])

print(train_df[["city_full", "encoded_city"]].head())


                       city_full   encoded_city
0  Atlanta-Sandy Springs-Roswell  192627.058387
1                     Pittsburgh  131828.753854
2                     Pittsburgh  131828.753854
3        Boston-Cambridge-Newton  472075.429542
4                     Pittsburgh  131828.753854


In [9]:
# REMOVE UNUSED COLUMNS TO PREVENT DATA LEAKAGE
drop = ["date", "city_full", "zipcode", "city", "median_sale_price"]
train_df.drop(columns=drop, inplace=True)
eval_df.drop(columns=drop, inplace=True)

In [10]:
print(train_df.shape)
train_df.head(1)

(576815, 40)


,year,month,day,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,avg_sale_to_list,sold_above_list,off_market_in_two_weeks,bank,bus,hospital,mall,park,restaurant,school,station,supermarket,Total Population,Median Age,Per Capita Income,Total Families Below Poverty,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,lat,lng,zipcode_freq,encoded_city
0,2012,3,31,217450.0,31.813674,110.183666,14.0,23.0,44.0,64.0,59.5,0.943662,0.142857,0.043478,12.0,2.0,4.0,1.0,60.0,45.0,57.0,4.0,7.0,5811.0,36.3,33052.0,5811.0,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,200773.999557,33.7338,-84.3922,94,192627.058387


In [11]:
train_df.to_csv("/Users/larry/house-price-regression/data/processed/train_engineered.csv", index=False)
eval_df.to_csv("/Users/larry/house-price-regression/data/processed/eval_engineered.csv", index=False)